# PCA降维
### 1、用途：降维中最常用的一种手段
### 2、目标：提取最有价值的目标（基于方差），LDA基于分类（寻找使不同类别间隔最大的降维方式）
### 3、降维方式：无监督的方式，无需知道标签。通过基变换进行降维，那么寻找最优单位基向量（单位向量）是一个问题
## 如何进行PCA降维？
假设有(3,5)的数据，一行表示一条数据，一列表示一个特征：
$$ X = \left[ \begin{array} {data}
2 & 5 & -1 & -3 & 12\\
6 & 8 & 9 & 12 & -6\\
3 & 7 & 5 & -4 & -11\\
\end{array} \right] $$

#### 1.将数据中心化，求协方差矩阵C（<font color='red'>特征是主要目标</font>）。
抽象的假设数据集 $ D = \left[ \begin{array} {data}
a_1 & a_2 & a_3 & ... & a_m\\
b_1 & b_2 & b_3 & ... & b_m\\
\end{array} \right] $，一个简单计算协方差矩阵的方式为：$\frac{1}{m}DD^T = \left[ \begin{array} {data}
\frac{1}{m}\sum_{i=1}^{m}a_i^2 & \frac{1}{m}\sum_{i=1}^{m}a_ib_i\\
\frac{1}{m}\sum_{i=1}^{m}a_ib_i & \frac{1}{m}\sum_{i=1}^{m}b_i^2\\
\end{array} \right] $
<br>
注：协方差的意义：样本的X高于均值, 一般Y也高于均值。相反如果X低于均值Y也低于均值。所以数据中心化的目的就是为了方便观察特征A、B协方差的关系，若是正相关：中心化后的cov(A,B)>0；若负相关：中心化后cov(A,B)< 0。
#### 2.将协方差矩阵C对角化，求出特征向量和特征值
由1可知，C是一个对角矩阵，对角线为各个特征的方差，非对角线位置为各个特征的协方差。而PCA需要的是最大化方差（特征内部具有最大区分度）以及最小化协方差（各个特征之间无关），其实就是最大化C对角线值，以及使C的非对角线位置为0。 <br>
<b>线性代数知识：一个n行n列的实对称矩阵一定可以找到n个单位正交特征向量，使得该矩阵对角化。</b> <br>
那么取k个最大的特征值和其对应的特征向量，该特征向量作为基变换的基向量。
#### 3.利用基向量降维，相当于做坐标轴的变换（<font color='red'>每一条的数据是主要目标</font>）。

In [1]:
import numpy as np

In [2]:
#1.获取数据(一列表示一条数据，一行表示一个特征)
X = np.array([[2,5,-1,-3,12],[6,8,9,12,-6],[3,7,5,-4,-1]]).reshape(3,5)
print("数据集：")
X

数据集：


array([[ 2,  5, -1, -3, 12],
       [ 6,  8,  9, 12, -6],
       [ 3,  7,  5, -4, -1]])

In [3]:
feature_num,data_num = X.shape
print("特征数量：{0}，数据条数：{1}".format(feature_num,data_num))

特征数量：3，数据条数：5


In [4]:
#2.将每个特征中心化
centerized_X = X-X.sum(axis=1).reshape(feature_num,1)
print("中心化后的数据集：")
centerized_X

中心化后的数据集：


array([[-13, -10, -16, -18,  -3],
       [-23, -21, -20, -17, -35],
       [ -7,  -3,  -5, -14, -11]])

In [5]:
#3.计算协方差矩阵
C = np.dot(centerized_X,centerized_X.T)
C

array([[ 858, 1240,  486],
       [1240, 2884,  947],
       [ 486,  947,  400]])

In [6]:
#4.协方差矩阵的特征值以及对应特征向量
eigenvalue,featurevector=np.linalg.eig(C)
print("C的特征值：\n{0}\n特征向量：\n{1}".format(eigenvalue,featurevector))

C的特征值：
[3804.6634843   274.65837262   62.67814309]
特征向量：
[[ 0.41159125  0.87258305 -0.2630427 ]
 [ 0.86117249 -0.4668353  -0.20111376]
 [ 0.29828607  0.14374847  0.94358985]]


In [7]:
#5.选择k个最大的特征特征向量为基向量，这里将3维特征降到2维特征
k = 2
k_max_featureValues_index = np.argsort(eigenvalue)[::-1][:k]
k_max_featureVec = featurevector[k_max_featureValues_index]
print("最大的特征向量为：\n{0}".format(k_max_featureVec))

最大的特征向量为：
[[ 0.41159125  0.87258305 -0.2630427 ]
 [ 0.86117249 -0.4668353  -0.20111376]]


In [8]:
#6.将原数据X进行基变换
k_max_featureVec.shape
mapped_X = np.dot(k_max_featureVec,X)
print("成功将形状为{0}映射形状为{1},特征维度从{2}变成{3}".format(X.shape,mapped_X.shape,X.shape[0],mapped_X.shape[0]))

成功将形状为(3, 5)映射形状为(2, 5),特征维度从3变成2


In [9]:
#7.数据对比
print("--------映射前---------")
print(X)
print("--------映射后---------")
print(mapped_X)

--------映射前---------
[[ 2  5 -1 -3 12]
 [ 6  8  9 12 -6]
 [ 3  7  5 -4 -1]]
--------映射后---------
[[ 5.26955273  7.19732181  6.12644272 10.28839362 -0.03336056]
 [-1.6820081  -0.83661626 -6.06825899 -7.38108604 13.33619545]]
